In [ ]:
from gerrychain.random import random

SEED = 0
while (SEED % 2 == 0):
    SEED = random.randrange(1000000001,9999999999)
#SEED = 5558870033
random.seed(SEED)
print(SEED)


import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election, tree)
#from gerrychain.proposals import recom
from gerrychain.tree import (recursive_tree_part, bipartition_tree)
from functools import partial
import pandas
import geopandas as gp
import time
import csv
import os

In [ ]:
# Read in shapefile with geopandas

shapefile_path = 'IL_tracts_projected32616_MOD_SingleParts_Data_Plan/IL_tracts_projected32616_MOD_SingleParts_Data_Plans.shp'

df = gp.read_file(shapefile_path)

# Create Graph object from gerrychain
graph = Graph.from_geodataframe(df)

# print(graph.nodes.data())
# for node in graph.nodes:
#     print(graph.nodes[node]['GEOID20'])

# print(graph.edges.data())



# outAdj = open('IL_BGs_projected32616_MOD_SingleParts_AUGAdj_outside.csv','w')
# writerAdj = csv.writer(outAdj,delimiter=',')

# writerAdj.writerow(['Unit 1','Unit 2','Shared perim (meters)'])
# for edge in graph.edges:
#     writerAdj.writerow([str(graph.nodes[edge[0]]['GEOID20']),str(graph.nodes[edge[1]]['GEOID20']),str(graph.edges[edge]['shared_perim'])])
    
# for node in graph.nodes:
#     if graph.nodes[node]['boundary_node']:
#         writerAdj.writerow([str(graph.nodes[node]['GEOID20']),'outside',graph.nodes[node]['boundary_perim']])
    
    
# outAdj.close()

In [ ]:
# FUNCTIONS --------------------------------------------------------------------------------------------

# Input: partition object
# Output: number of majority-minority districts that are plurality Black, Lat/Hisp, and nh-white
def calculate_MM_districts(my_partition):
    
    plur_B = 0
    plur_L = 0
    plur_W = 0
    
    for district in my_partition['nhwhitepop']:
    
        fraction_W = my_partition['nhwhitepop'][district]/my_partition['population'][district]
        fraction_B = my_partition['blackpop'][district]/my_partition['population'][district]
        fraction_L = my_partition['latpop'][district]/my_partition['population'][district]

        if fraction_W < 0.5:
            if fraction_B > max(fraction_W,fraction_L):
                plur_B += 1
            elif fraction_L > max(fraction_W,fraction_B):
                plur_L += 1
            else:
                plur_W += 1
            
    return plur_B,plur_L,plur_W


# Input: partition object
# Output: number of competitive districts (within 'margin' of victory) in that partition
def calculate_Cmpttv_districts(my_partition, margin):
    
    count_cmpttv = 0
    
    for dem_percent in my_partition['AVG'].percents('Dem'):
        if abs(dem_percent-(1-dem_percent)) < margin:
            count_cmpttv += 1
            
    return count_cmpttv


# Modified from GerryChain package:
# https://github.com/mggg/GerryChain/blob/main/gerrychain/proposals/tree_proposals.py
def recom(my_partition, pop_col, pop_target, epsilon, node_repeats=1, method=bipartition_tree):
    
    edge = random.choice(tuple(my_partition['cut_edges']))
    parts_to_merge = (my_partition.assignment[edge[0]], my_partition.assignment[edge[1]])
    #print('parts_to_merge: ',parts_to_merge)

    subgraph = my_partition.graph.subgraph(
        my_partition.parts[parts_to_merge[0]] | my_partition.parts[parts_to_merge[1]]
    )
    
    # This is the modified part
    # Need to modify target population and epsilon based on the two districts chosen
    
    new_pop_target = (my_partition['population'][parts_to_merge[0]] + my_partition['population'][parts_to_merge[1]])/2
    new_epsilon = min(abs(1-(((1+epsilon)*pop_target)/new_pop_target)),abs(1-(((1-epsilon)*pop_target)/new_pop_target)))
    

    flips = recursive_tree_part(
        subgraph,
        parts_to_merge,
        pop_col=pop_col,
        pop_target=new_pop_target,
        epsilon=new_epsilon,
        node_repeats=node_repeats,
        method=method,
    )

    return my_partition.flip(flips)


# Input: partition object, constraint/threshold dictionary, objective string
# Output: partition object
# Proposal function returns new partition, new partition checked for constraints and objective improvement
# If new partition passes checks, return new partition. Else, return original partition.
def ReComMove(my_partition,constraint_dict,obj,margin):
    
    proposed_next_state = proposal(my_partition)
    
    checks = True
    why = ''
    
    # Check population balance
#     popDev = max([abs(1-(p/ideal_population))for p in proposed_next_state['population'].values()])
#     if popDev > max_population_dev:
#         checks = False
    
    # Check constraints
    if checks and ('demo' in constraint_dict):
        plur_B,plur_L,plur_W = calculate_MM_districts(proposed_next_state)
        #if plur_B != num_plurality_B or plur_L != num_plurality_L or plur_W != num_plurality_W:
        if plur_B != constraint_dict['demo']['plurB'] or plur_L != constraint_dict['demo']['plurL'] or plur_W != constraint_dict['demo']['plurW']:
            checks = False
            why = 'demo'
                
    if checks and ('perim' in constraint_dict):
        if sum(proposed_next_state['perimeter'].values()) > constraint_dict['perim']:
            checks = False
            why = 'perim'
            
    if checks and ('cmpttv' in constraint_dict):
        if calculate_Cmpttv_districts(proposed_next_state,margin) != calculate_Cmpttv_districts(my_partition,margin):
            checks = False
            why = 'cmpttv'
            
    if checks and ('eg' in constraint_dict):
        if abs(proposed_next_state['AVG'].efficiency_gap()) > constraint_dict['eg']:
            checks = False
            why = 'eg'
            
    if checks and ('mm' in constraint_dict):
        if abs(proposed_next_state['AVG'].mean_median()) > constraint_dict['mm']:
            checks = False
            why = 'mm'
            
            
    # Check objective
    if checks:
        if obj == 'perim':
            if sum(proposed_next_state['perimeter'].values()) > sum(my_partition['perimeter'].values()):
                checks = False
                why = 'perim'
                
        elif obj == 'eg':
            if abs(proposed_next_state['AVG'].efficiency_gap()) > abs(my_partition['AVG'].efficiency_gap()):
                checks = False
                why = 'eg'
                
        elif obj == 'mm':
            if abs(proposed_next_state['AVG'].mean_median()) > abs(my_partition['AVG'].mean_median()):
                checks = False
                why = 'mm'
                
        elif obj == 'cmpttv':
            if calculate_Cmpttv_districts(proposed_next_state,margin) < calculate_Cmpttv_districts(my_partition,margin):
                checks = False
                why = 'cmpttv'

    
    if checks:
        #print('Success')
        return proposed_next_state,why
    else:
        #print('Fail')
        return my_partition,why



# SET-UP ----------------------------------------------------------------------------------------------------


# District plan files
PlanFolderPath = 'InitialPlans_IL_Tracts'

#JustFolderName = PlanFolderPath.split('/')

maps = [p for p in os.listdir(PlanFolderPath+'/') if not p.startswith('.')]
maps.sort()

CURRENT = '2021_IL_CongressionalMap_TractApprox_popBal1per.csv'
NEUTRAL1 = 'ReCom_IL_tracts_5832966053Seed_25It_0.69minutes_noneObj_demo_Perim10731.637990858299.csv'
NEUTRAL2 = 'ReCom_IL_tracts_5832966053Seed_25It_0.13minutes_noneObj_demo_Perim9326.075811435445.csv'
NEUTRAL3 = 'ReCom_IL_tracts_5832966053Seed_25It_0.09minutes_noneObj_demo_Perim9454.520800199896.csv'

map_to_run = CURRENT

# List specific maps in PlanFolderPath to run
maps = [map_to_run]


# Set parameters

output_path = 'Test'

unit_level = 'tracts'
#unit_level = 'BGs'

objective = 'perim'
#objective = 'eg'
#objective = 'mm'
#objective = 'cmpttv'
#objective = 'none'

# Set average obj. value from Flip runs to see when ReCom runs reach it
flip_avg = 0.1354

# Choose constraints
if objective == 'perim':
    constraints = {'demo':{}}
elif objective != 'none':
    constraints = {'demo':{}, 'perim':2000000}

# constraints = {'demo': {}, 'perim': value to be added to initial perimeter (give in meters!),
#                'eg': Efficiency Gap threshold, 'mm': Mean-Median threshold,
#                'cmpttv': 0 (nothing required)}


max_population_dev = 0.01
num_node_repeats = 1
cmpttv_margin = 0.07


iterations = 200
converge = True
convergence_it = 200
convergence_threshold = 2 # Perim
#convergence_threshold = 0.0001 # EG, MM, Cmpttv
repeated_runs = 1




# Create Election object from gerrychain

election = Election('AVG', {'Dem': 'VOTES_DEM', 'Rep': 'VOTES_REP'})


# Open output file to link initial map with optimized map

# outMap = open(output_path + '/ReCom_IL_'+unit_level+'_'+objective+'_'+str(SEED)+'Seed_MapList.csv','w')
# writerMap = csv.writer(outMap,delimiter=',')
# writerMap.writerow(['Initial Map',objective+' Final Map'])

outRejects = open(output_path+'/ReCom_'+unit_level+'_'+objective+'Obj_'+str(SEED)+'Seed_Rejections.csv','w')
writerRejects = csv.writer(outRejects,delimiter=',')
reasons = ['demo','perim','eg','mm','cmpttv']
writerRejects.writerow(['Final Plan']+reasons+['Reach Flip'])


for i in range(0,repeated_runs):
    
    for file in maps:
        
        # Read in district plan
        planFile = open(PlanFolderPath + '/' + file,'r')
        reader = csv.reader(planFile,delimiter=',')

        labels = next(reader)
        plan = {}
        for line in reader:
            if line != [] and line != ['','']:
                plan[line[0]] = line[1]

        planFile.close()
        #print(plan)


        # Make plan dictionary for nodes in graph object
        plan_nodes = {}

        for node in graph.nodes:
            plan_nodes[node] = plan[graph.nodes[node]['GEOID20']]

        #print(plan_nodes)


        # Create a GeographicPartition object from gerrychain (initial district plan)
        # GeographicPartition automatically has updaters for perim, area, cut-edges, and more!

        initial_partition = GeographicPartition(
            graph,
            assignment = plan_nodes,
            updaters = {
                'population': updaters.Tally('POP20', alias='population'),
                'nhwhitepop': updaters.Tally('NHWHITEPOP', alias='nhwhitepop'),
                'blackpop': updaters.Tally('BLACKPOP', alias='blackpop'),
                'latpop': updaters.Tally('HISPLATPOP', alias='latpop'),
                'AVG': election
            }
        )


        # Set the ideal population
        ideal_population = sum(initial_partition['population'].values())/len(initial_partition)

        # Calculate number of majority-minority districts
        if 'demo' in constraints:
            num_plurality_B, num_plurality_L, num_plurality_W = calculate_MM_districts(initial_partition)
            constraints['demo'] = {'plurB':num_plurality_B, 'plurL':num_plurality_L, 'plurW':num_plurality_W}
            #print(num_plurality_B, num_plurality_L, num_plurality_W)

        # Calculate perimeter threshold
        if 'perim' in constraints:
            perim_threshold = sum(initial_partition['perimeter'].values()) + constraints['perim']
            constraints['perim'] = perim_threshold


        # Record objective values

        if objective == 'perim':
            objective_values = [sum(initial_partition['perimeter'].values())]

        elif objective == 'eg':
            objective_values = [initial_partition['AVG'].efficiency_gap()]

        elif objective == 'mm':
            objective_values = [initial_partition['AVG'].mean_median()]

        elif objective == 'cmpttv':
            objective_values = [calculate_Cmpttv_districts(initial_partition,cmpttv_margin)]


        # Build proposal function
        proposal = partial(recom,
                           pop_col='POP20',
                           pop_target=ideal_population,
                           epsilon=max_population_dev,
                           node_repeats=num_node_repeats
                          )


        print('Seed: ',SEED)

        if converge:
            print('Iterations: until convergence')
        else:
            print('Iterations: ',iterations)

        print('Initial max pop dev: ',max([abs(1-(p/ideal_population))for p in initial_partition['population'].values()]))

        if objective != 'perim':
            print('Initial perim: ',sum(initial_partition['perimeter'].values())/1000)

        if objective != 'none':
            if objective == 'perim':
                print('Initial ' + objective + ': ',objective_values[-1]/1000)
            else:
                print('Initial ' + objective + ': ',objective_values[-1])



# ITERATIONS ------------------------------------------------------------------------------------------------

        start = time.time()

        # Perform iterations
        partition_OLD = initial_partition
        k = 0
        finished = False
        numIt = 0
        
        countRejects = {}
        for re in reasons:
            countRejects[re] = 0
            
        time_reached_Flip = -1

        while not finished:

            k += 1
            if converge and k >= 2:
                if (abs(objective_values[-2] - objective_values[-1]) < convergence_threshold):
                    numIt += 1
                else:
                    numIt = 0

                if numIt >= convergence_it:
                    finished = True
                    print(k)
                    continue

            elif k >= iterations:
                finished = True


            if k % 100 == 0:
                print(k)

            # Get new partition
            partition_NEW,whyRejected = ReComMove(partition_OLD, constraints, objective, cmpttv_margin)

            if whyRejected != '':
                countRejects[whyRejected] += 1

            # Record new objective value
            if objective == 'perim':
                objective_values.append(sum(partition_NEW['perimeter'].values()))

            elif objective == 'eg':
                objective_values.append(partition_NEW['AVG'].efficiency_gap())

            elif objective == 'mm':
                objective_values.append(partition_NEW['AVG'].mean_median())

            elif objective == 'cmpttv':
                objective_values.append(calculate_Cmpttv_districts(partition_NEW,cmpttv_margin))
                
            
            if objective == 'cmpttv':
                if objective_values[-1] >= flip_avg and time_reached_Flip < 0:
                    time_reached_Flip = time.time() - start
            elif objective != 'none':
                if abs(objective_values[-1]) <= flip_avg and time_reached_Flip < 0:
                    time_reached_Flip = time.time() - start


            # New partition becomes old partition
            partition_OLD = partition_NEW



# OUTPUT -----------------------------------------------------------------------------------------------------


        if objective == 'perim':    
            objective_values = [p/1000 for p in objective_values]


        # Plot objective values
        if objective != 'none':
            plt.plot(objective_values)
            plt.show()
            print('Final ',objective,' = ',objective_values[-1])


        maxDev = max([abs(1-(p/ideal_population))for p in partition_OLD['population'].values()])
        print('Final max pop dev = ',maxDev)

        finalPerim = sum(partition_OLD['perimeter'].values())/1000
        if objective != 'perim':
            print('Final perim = ',finalPerim)
            

#         mapTime = round((time.time()-start)/60,2)
#         print('Runtime = ',mapTime,' minutes')

        mapTime = time.time()-start
        print('Runtime = ',mapTime,' seconds')


        # Output optimized map

        constraint_str = ''
        for c in constraints:
            constraint_str += c

        if objective != 'none':
            #outFile_str = 'ReCom_IL_'+unit_level+'_'+str(SEED)+'Seed_'+str(k)+'It_'+str(mapTime)+'minutes_'+str(objective)+'Obj'+str(round(objective_values[-1],5))+'_'+constraint_str+'_Perim'+str(finalPerim)+'.csv'
            outFile_str = 'ReCom_IL_'+unit_level+'_'+str(SEED)+'Seed_'+str(k)+'It_'+str(mapTime)+'seconds_'+str(objective)+'Obj'+str(round(objective_values[-1],5))+'_'+constraint_str+'_Perim'+str(finalPerim)+'.csv'
            outFile = open(output_path + '/' + outFile_str,'w')
        else:
            #outFile_str = 'ReCom_IL_'+unit_level+'_'+str(SEED)+'Seed_'+str(k)+'It_'+str(mapTime)+'minutes_'+str(objective)+'Obj_'+constraint_str+'_Perim'+str(finalPerim)+'.csv'
            outFile_str = 'ReCom_IL_'+unit_level+'_'+str(SEED)+'Seed_'+str(k)+'It_'+str(mapTime)+'seconds_'+str(objective)+'Obj_'+constraint_str+'_Perim'+str(finalPerim)+'.csv'
            outFile = open(output_path + '/' + outFile_str,'w')
            
        writer = csv.writer(outFile,delimiter=',')

        writer.writerow(['GEOID20','district'])
        for node in list(partition_OLD.graph.nodes):
            writer.writerow([graph.nodes[node]['GEOID20'],partition_OLD.assignment[node]])

        outFile.close()
        
        writerRejects.writerow([outFile_str]+[countRejects[val] for val in countRejects]+[time_reached_Flip])
        
        
        #writerMap.writerow([file,outFile_str])
        
#outMap.close()
outRejects.close()
